In [ ]:
import threading

# Helper thread to avoid the Spark StreamingContext from blocking Jupyter
        
class StreamingThread(threading.Thread):
    def __init__(self, ssc):
        super().__init__()
        self.ssc = ssc
    def run(self):
        self.ssc.start()
        self.ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql.session import SparkSession

# #Spark Config
conf = SparkConf().setAppName("sample_app")
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

In [ ]:
socketDF = spark.readStream.format("socket").option("host", "localhost").option("port", 8080).load()
socketDF.printSchema()

In [ ]:
from pyspark.sql.functions import from_json, schema_of_json

In [ ]:
def process_row(df, epoch_id):
    print(epoch_id)
    if df.count() == 0: return
    schema = schema_of_json(df.first().value)
    df_cols = df.selectExpr('CAST(value AS STRING)')\
        .select(from_json('value', schema)\
        .alias('temp'))\
        .select('temp.*')
    df_cols.show()

In [ ]:
query = socketDF.writeStream.trigger(processingTime='5 seconds').foreachBatch(process_row).start()  

In [ ]:
query.stop()